<a href="https://colab.research.google.com/github/kislay960/Emojify-it/blob/main/emoify_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install emoji

     |████████████████████████████████| 51kB 4.5MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=676750cce95a21dbd0f6dca3db2b54a5706ceecbb3df1001ff12a9f0834aa828
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [2]:
import numpy as np
import pandas as pd

In [3]:
train = pd.read_csv('https://raw.githubusercontent.com/kislay960/Data-Science-Practice/main/train_emoji.csv',header=None)
test = pd.read_csv('https://raw.githubusercontent.com/kislay960/Data-Science-Practice/main/test_emoji.csv',header=None)

In [4]:
import emoji

In [5]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

In [6]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [7]:
data = train.values
for i in range(10):
    print(data[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😓
congratulations on your acceptance 😁
The assignment is too long  😓
I want to go play ⚾


In [8]:
from keras.utils import to_categorical

In [9]:
XT = train[0]
Xt = test[0]

YT = to_categorical(train[1])
Yt = to_categorical(test[1])

print(XT.shape)
print(Xt.shape)
print(YT.shape)
print(Yt.shape)

(132,)
(56,)
(132, 5)
(56, 5)


In [11]:
embeddings = {}
with open('/content/drive/MyDrive/glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        
        #print(word)
        #print(coeffs)
        embeddings[word] = coeffs

In [12]:
def getOutputEmbeddings(X):
    
    embedding_matrix_output = np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx] = embeddings[X[ix][jx].lower()]
            
    return embedding_matrix_output

In [13]:
emb_XT = getOutputEmbeddings(XT)
emb_Xt = getOutputEmbeddings(Xt)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [14]:
print(emb_XT.shape)
print(emb_Xt.shape)

(132, 10, 50)
(56, 10, 50)


In [15]:
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.models import Sequential

In [16]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 64)            29440     
_________________________________________________________________
dropout (Dropout)            (None, 10, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
_________________________________________________________________
activation (Activation)      (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
____________________________________________________

In [17]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
model.fit(emb_XT,YT,batch_size=32,epochs=40,shuffle=True,validation_split=0.1)

Epoch 1/40
4/4 [==============================] - 1s 203ms/step - loss: 1.5503 - acc: 0.4068 - val_loss: 1.6354 - val_acc: 0.3571
Epoch 2/40
4/4 [==============================] - 0s 14ms/step - loss: 1.5251 - acc: 0.3220 - val_loss: 1.6968 - val_acc: 0.0714
Epoch 3/40
4/4 [==============================] - 0s 15ms/step - loss: 1.4741 - acc: 0.3644 - val_loss: 1.6685 - val_acc: 0.3571
Epoch 4/40
4/4 [==============================] - 0s 15ms/step - loss: 1.4542 - acc: 0.4153 - val_loss: 1.6348 - val_acc: 0.3571
Epoch 5/40
4/4 [==============================] - 0s 14ms/step - loss: 1.3843 - acc: 0.4576 - val_loss: 1.5624 - val_acc: 0.2857
Epoch 6/40
4/4 [==============================] - 0s 13ms/step - loss: 1.3343 - acc: 0.5254 - val_loss: 1.4982 - val_acc: 0.2857
Epoch 7/40
4/4 [==============================] - 0s 14ms/step - loss: 1.2624 - acc: 0.5339 - val_loss: 1.4309 - val_acc: 0.4286
Epoch 8/40
4/4 [==============================] - 0s 14ms/step - loss: 1.1752 - acc: 0.6186 - va

In [18]:
model.evaluate(emb_Xt,Yt)

2/2 [==============================] - 0s 4ms/step - loss: 1.4902 - acc: 0.6071


[1.4901554584503174, 0.6071428656578064]

In [19]:
pred = model.predict_classes(emb_Xt)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [20]:
for i in range(30):
    print(' '.join(Xt[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Yt[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😓
😓
he got a raise
😁
😓
she got me a present
❤️
😓
ha ha ha it was so funny
😁
😁
he is a good friend
❤️
❤️
I am upset
❤️
😓
We had such a lovely dinner tonight
❤️
😁
where is the food
🍴
🍴
Stop making this joke ha ha ha
😁
😓
where is the ball
⚾
⚾
work is hard
😓
😁
This girl is messing with me
😓
❤️
are you serious ha ha
😁
😓
Let us go play baseball
⚾
⚾
This stupid grader is not working
😓
😓
work is horrible
😓
😓
Congratulation for having a baby
😁
😁
stop messing around
😓
😓
any suggestions for dinner
🍴
🍴
I love taking breaks
❤️
❤️
you brighten my day
😁
❤️
I boiled rice
🍴
🍴
she is a bully
😓
❤️
Why are you feeling bad
😓
😓
I am upset
😓
😓
I worked during my birthday
😓
😁
My grandmother is the love of my life
❤️
❤️
enjoy your break
😁
⚾
valentine day is near
❤️
😁


In [21]:
with open("model.json", "w") as file:
    file.write(model.to_json())
model.save_weights("model.h5")

##Code to be used in django

In [22]:
from keras.models import model_from_json

In [23]:
with open("model.json", "r") as file:
    model = model_from_json(file.read())
model.load_weights("model.h5")

In [24]:
test_str = "Hello how are you"
X = pd.Series([test_str])

emb_X = getOutputEmbeddings(X)

p = model.predict_classes(emb_X)

print(' '.join(X[0]))
print(emoji.emojize(emoji_dictionary[str(p[0])]))

Hello how are you
❤️
